In [ ]:
!pip install -q --upgrade keras-nlp tensorflow #For Keras-NLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 83.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

import math
import io
import re
import string
import tqdm

import keras_nlp

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
!wget http://www.cs.biu.ac.il/~koppel/blogs/blogs.zip

In [ ]:
!unzip blogs.zip

In [ ]:
!pip install lxml
!pip install bs4

In [ ]:
from bs4 import BeautifulSoup as bs


In [ ]:

contents=[]
for f in tqdm.tqdm(os.listdir('blogs')):
  with open(f'blogs/{f}', 'rb') as file_:
    lines=file_.read()
    contents.append(bs(lines, "lxml").text)

In [ ]:
text='\n'.join(contents[:2000])

In [ ]:
with open('temp.txt','w') as f:
  f.write(text)

In [ ]:
# path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
ngrams=5
path_to_file='temp.txt',
text=tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))
text=text.filter(lambda x:len(tf.strings.split(x," "))>ngrams-1)
# text=text.map(lambda x:tf.strings.unicode_decode(x, 'UTF-8'))

In [ ]:
vocab_size=16384
tokenizer=tf.keras.preprocessing.text.Tokenizer(
    num_words=vocab_size,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None,
    analyzer=None,
)

In [ ]:
texts=list(map(lambda x: x.decode("utf-8"),list(text.as_numpy_iterator())))

In [ ]:
tokenizer.fit_on_texts(texts)

In [ ]:
sequences=tokenizer.texts_to_sequences(texts)

In [ ]:
new_sequences=[]
for seq in sequences:
  l=len(seq)
  # for i in range(2,ngrams):
  #   new_sequences.append(seq[:i])
  for i in range(ngrams,l):
    new_sequences.append(seq[i-ngrams:i])

new_sequences = np.array(pad_sequences(new_sequences, maxlen=5, padding='pre'))
new_sequences_x= new_sequences[:,:-1]
new_sequences_y= new_sequences[:,1:]
l=len(new_sequences)
thr=int(l*0.7)

In [ ]:
data_train=tf.data.Dataset.from_tensor_slices((new_sequences_x[:thr],new_sequences_y[:thr]))
data_test=tf.data.Dataset.from_tensor_slices((new_sequences_x[thr:],new_sequences_y[thr:]))

In [ ]:
inp=tf.keras.layers.Input((ngrams-1,))
embedding=tf.keras.layers.Embedding(vocab_size, 64)(inp)
# att1=tf.keras.layers.MultiHeadAttention(num_heads=3,key_dim=32,output_shape=64)(embedding,embedding)
# att2=tf.keras.layers.MultiHeadAttention(num_heads=3,key_dim=32,output_shape=64)(att1,att1)
lstm1=tf.keras.layers.LSTM(128,return_sequences=True)(embedding)
lstm2=tf.keras.layers.LSTM(128,return_sequences=True)(lstm1)
# features1=tf.keras.layers.GlobalAveragePooling1D()(att2)
# flatten=tf.keras.layers.Flatten()(lstm2)
out=tf.keras.layers.Dense(vocab_size, activation=tf.keras.layers.Softmax(axis=-1))(lstm2)
model=tf.keras.Model(inputs=inp,outputs=out)
model.summary()

In [ ]:
y_train=new_sequences_y

In [ ]:
y_train=np.array(y_train)[:,-1]

In [ ]:
np.unique(y_train)

array([    1,     2,     3, ..., 16381, 16382, 16383], dtype=int32)

In [ ]:
class_weight={}
for i in tqdm.tqdm(range(vocab_size)):
  freq=(np.where(y_train==i,1,0).sum()+1)/len(y_train)
  freq=math.pow(freq,0.66)
  class_weight[i]=1/freq
  if class_weight[i]>30:
    class_weight[i]=30

100%|██████████| 16384/16384 [01:15<00:00, 216.26it/s]


In [ ]:
model.compile(optimizer=tf.optimizers.RMSprop(learning_rate=0.001),loss=tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=False,
        ),metrics=["accuracy",keras_nlp.metrics.Perplexity()])

checkpoint_filepath = 'temp.ckpt'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


In [ ]:
model.fit(data_train.batch(32),epochs=128,validation_data=data_test.batch(32),callbacks=[model_checkpoint_callback],class_weight=class_weight)

In [ ]:
out=model.predict(new_sequences_x)

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
def predict_next_word(input_sequence):
  seq=tokenizer.texts_to_sequences([input_sequence])[0]
  if len(seq)>4:
    seq=seq[-4:]
    l=3
  else:
    l=len(seq)-1
    seq=pad_sequences([seq], maxlen=ngrams-1, padding='post')[0]
  seq=list(seq)
  seq.append(model.predict(np.array([seq]))[0].argmax(axis=-1)[l])
  if len(seq)>4:
    seq=seq[-4:]
  seq_inv=tokenizer.sequences_to_texts([np.array(seq)])[0]
  input_sequence+=(" "+seq_inv.split(" ")[-1])
  return input_sequence

In [ ]:
seq="you have seen"
for i in range(20):
  seq=predict_next_word(seq)
print(seq)

In [ ]:
(new_sequences_x>16382).sum()

In [ ]:
new_sequences_x.shape